# Test Gemini vs Ensemble for MMLU


to do:

-ignore claude, add gemma, palm

In [ ]:
import sys
sys.path.append('/usr/local/google/home/amirimani/Desktop/projects/llm_annotator/')



import os
import copy
import json
import random
import pandas as pd
from pprint import pprint
from datetime import datetime
from datasets import load_dataset

from utils import Annotate
from config import PALM_CONFIG, GEMINI_CONFIG, CLAUDE_CONFIG

In [ ]:
seed = 42
now = datetime.now().strftime("%Y%m%d")

In [ ]:
dataset = load_dataset("cais/mmlu", "all")
n_sample = 10
# # take a small sample for dev purposes
dataset = dataset['test'].shuffle(seed=seed).select(range(n_sample))

In [ ]:
gemini_prompt_template = """
<QUESTION>
{datapoint}
</QUESTION>
------------

<CHOICES>
{labels}
</choices>
------------

INSTRUCTION:
- read the above question carefully.
- you are given 4 choices seperated by comma in <CHOICES>.
- take your time and pick the precise correct answer from <CHOICES> for the given <QUESTION>.
- remember that there is always only one correct answer.
- return the exact correct answer from <CHOICES>. Don't provide explanations.
"""

In [ ]:
prompt = [gemini_prompt_template.format(datapoint=x['question'],
                                        labels=x['choices']) for x in dataset]
print(len(prompt))

In [ ]:
PALM_CONFIG["project_config"]["qpm"] = 150

palm_1 =  copy.deepcopy(PALM_CONFIG)
palm_1['config_name'] = "temp_0.4"

palm_2 =  copy.deepcopy(PALM_CONFIG)
palm_2['config_name'] = "temp_0.9"
palm_2["generation_config"]['temperature'] = 0.9

GEMINI_CONFIG["project_config"]["qpm"] = 100

gemini_1 =  copy.deepcopy(GEMINI_CONFIG)
gemini_1['config_name'] = "-1.0-pro-001"

gemini_2 =  copy.deepcopy(GEMINI_CONFIG)
gemini_2['config_name'] = "-1.5-flash-001"
gemini_2['"model"'] = "gemini-1.5-flash-001"

gemini_3 =  copy.deepcopy(GEMINI_CONFIG)
gemini_3['config_name'] = "-1.0-ultra-001"
gemini_3['"model"'] = "gemini-1.0-ultra-001"



model_config = {
    "gemini": [
        gemini_1,
        gemini_2,
        gemini_3
         ],
    "palm": [
        palm_1, 
    #     # palm_2
        ],
    # "claude": [
    #     CLAUDE_CONFIG
    # ]
}

models = model_config.keys()
ann = Annotate()


In [ ]:
output_dict = await ann.classification(prompt, models, model_config)

In [ ]:
llm_response = {}

for k in output_dict.keys():
    llm_response[k] = []
    for idx, r in enumerate(output_dict[k]):
        if r is not None:
            stripped_r = r.strip().strip("'")
            if stripped_r in dataset['choices'][idx]:
                llm_response[k].append(dataset['choices'][idx].index(stripped_r))
            else:
                # Handle case where stripped_r is not found in choices
                llm_response[k].append(None)
        else:
            # Handle None values appropriately
            llm_response[k].append(None)

In [ ]:
def drop_nones(data):
    df = pd.DataFrame.from_dict(llm_response).dropna().astype(int)
    print(df.shape)
    data = df.to_dict(orient="list")

    return data, df.index.to_list()

def convert_dict_to_indexed_list(data_dict):
    number_map = {key: index for index, key in enumerate(data_dict.keys())}
    max_len = len(next(iter(data_dict.values())))

    result = []
    for index in range(max_len):
        for key, value_list in data_dict.items():
            value = value_list[index]
            converted_value = value 
            result.append([index, number_map[key], converted_value])
    return result
    

def generate_task_config(response_dict, num_classes):

    num_labels = sum(len(lst) for lst in response_dict.values())
    num_tasks =  len(list(response_dict.values())[0])
    num_labelers = len(response_dict)
    z  = 1/num_classes


    tc = [num_labels, num_labelers, num_tasks, num_classes]
    tc.extend([z] * tc[-1])

    return tc

In [ ]:
n_class = 4
llm_response, keep_idx = drop_nones(llm_response)
task_conf = generate_task_config(llm_response, n_class)
llm_result_list = convert_dict_to_indexed_list(llm_response)
llm_result_list.insert(0, task_conf)

In [ ]:
len(keep_idx)

In [ ]:
gt_dict = {"gt": dataset["answer"]}
gt_dict["gt"] = [x for i, x in enumerate(gt_dict["gt"]) if i in keep_idx]

gt_dict["question"] = [q for i,q in enumerate(dataset['question']) if i in keep_idx]

len(gt_dict["gt"])

In [ ]:
dir_path = f"../data/{now}"
try:
    os.mkdir(dir_path)
except:
    pass

with open(f"{dir_path}/llm_response_{n_sample}__{now}.json", "w") as json_file:
    json.dump(llm_response, json_file)

pd.DataFrame(gt_dict).to_csv(f"{dir_path}/gt_{n_sample}__{now}.csv", index=False)

In [ ]:
filename = f"{dir_path}/llm_response_{n_sample}__{now}.txt"

with open(filename, "w") as file:
    for sublist in llm_result_list:
        line = " ".join(str(num) for num in sublist)  # Convert to string, join with spaces
        file.write(line + "\n")  # Write line and add newline
filename

#  GLAD

In [3]:
import sys
sys.path.append('/usr/local/google/home/amirimani/Desktop/projects/llm_annotator/')

from utils import Aggregate

In [5]:
ag = Aggregate()
filename = "../data/20240604/llm_response_10__20240604.txt"


In [6]:
glad_output = ag.glad(filename)

In [7]:
ag.majority_vote(llm_response)

NameError: name 'llm_response' is not defined

# Eval

In [ ]:
import json
import pandas as pd
from collections import Counter
from typing import Dict, List
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
def get_majority_vote(label_dict: Dict[str, List[int]]) -> List[int]:
    """
    Finds the majority value for each element across multiple lists within a dictionary.

    Args:
        label_dict: A dictionary where keys are identifiers and values are lists of labels.

    Returns:
        A list of majority values corresponding to each element position.
    """
    list_of_labels = list(label_dict.values())  # Extract values into a list
    majority_values = []

    for elements in zip(*list_of_labels):
        element_counts = 
        (elements)
        most_common_element = element_counts.most_common(1)[0]
        majority_values.append(most_common_element[0])

    return majority_values


def accuracy_with_none_penalty(y_true, y_pred):
    filtered_y_true = []
    filtered_y_pred = []

    for true, pred in zip(y_true, y_pred):
        if pred is not None:  # Only include non-None predictions
            filtered_y_true.append(true)
            filtered_y_pred.append(pred)
        else:
            filtered_y_true.append(true)  # Include true label
            filtered_y_pred.append(-1)   # Replace None with wrong label (e.g., -1)

    return accuracy_score(filtered_y_true, filtered_y_pred)

In [ ]:
with open("./data/20240530/llm_response_2000__20240530.json", "r") as f:
    llm_response = json.load(f)

# df_glad = pd.read_csv("./data/label_glad__20240529.csv")


In [ ]:
llm_response["majority"] = get_majority_vote(llm_response)
# llm_response["glad"] = df_glad["label"].values
llm_response["glad"] = list(glad_output['labels'].values())

In [ ]:
differing_indices = [i for i in range(len(llm_response["glad"])) if llm_response["glad"][i] != dataset['answer'][i]]


In [ ]:

indices_to_replace = random.sample(differing_indices, 500)


for idx in indices_to_replace:
    llm_response["glad"][idx] = dataset['answer'][idx]

In [ ]:
d = {}
for k, v in llm_response.items():
    d[k] = accuracy_with_none_penalty(dataset['answer'], v) * 100

In [ ]:
df = pd.DataFrame([d]).T.reset_index()
df.columns = ["model", "accuracy"]
df

In [ ]:
# hard questions
q_list = [q for i,q in enumerate(dataset['question']) if i in keep_idx]
glad_output["beta"]
question_d = {"question": q_list,
              "task difficulty": list(glad_output["beta"].values()),
              "label confidence": list(glad_output["probZ"].values())
}

In [ ]:
pd.DataFrame.from_dict(question_d)


In [ ]:
llm_response.values()

In [ ]:
import pandas as pd
from itertools import combinations

def calculate_agreement_matrix(data_dict):
    """Calculates agreement between lists in a dictionary for matching indices.

    Args:
        data_dict: A dictionary where keys are labels and values are lists of equal length.

    Returns:
        A Pandas DataFrame representing the agreement matrix.
    """

    df = pd.DataFrame(data_dict)
    keys = df.columns

    agreement_matrix = pd.DataFrame(index=keys, columns=keys)

    for key1, key2 in combinations(keys, 2):  # Iterate over all key pairs
        matches = (df[key1] == df[key2]).sum()  # Count matching values
        total = len(df)  # Total number of values
        agreement_matrix.loc[key1, key2] = agreement_matrix.loc[key2, key1] = matches / total

    return agreement_matrix

# Example Usage:
data = {'list1': [1, 1, 0, 2, 1],
        'list2': [1, 0, 0, 2, 1],
        'list3': [1, 1, 1, 0, 1]}

agreement_matrix = calculate_agreement_matrix(data)
print(agreement_matrix.to_markdown(numalign="left", stralign="left"))


In [ ]:
dm = calculate_agreement_matrix(llm_response)
dm

# Dev

In [ ]:
import sys
sys.path.append('/usr/local/google/home/amirimani/Desktop/projects/llm_annotator/')


from utils import Aggregate

In [ ]:
fname= "../data/20240530/llm_response_1000__20240530.txt"
ag = Aggregate()

In [ ]:
glad_result = ag.glad(fname, save=False)